In [73]:
%reset -f
import numpy as np
from scipy.io import loadmat, savemat
from scipy.sparse import csr_matrix
from scipy.sparse import find
from numba import jit
import warnings
from joblib import Parallel, delayed
import os
import time

from PDSCH_Class import mcs
from CRC_encoding import nrCRCencode
from nrModulation import nrSymbolModulate, nrSymbolDemodulate
# from nrLDPC import nrLDPCencode
from nrLDPC import nrLDPCencode, nrLDPCdecode


In [74]:
NumIter = int(1e+2)
snrdB = -3.2
BGN = 2
Z = 52
maxIter = 20
loadbuffer = False
np.random.seed(0)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [75]:
def Indicator(rngseed, chIn, mcs):
    np.random.seed(rngseed)
# def Indicator(chIn, mcs):
    noise = (np.random.standard_normal(mcs.SignalSize) + 1j * np.random.standard_normal(mcs.SignalSize))/np.sqrt(2)*mcs.noiseStd
    chOut = chIn + noise
    LLR = nrSymbolDemodulate(chOut, mcs.noiseVar, mcs.modulation)
    dec = nrLDPCdecode(LLR,mcs.Z,mcs.Num1,mcs.maxIter,mcs.Hsize,mcs.RowNum1,mcs.RowWiseIdxPerLayer)
    return not np.array_equal(mcs.In,dec[:-mcs.LTB])

In [76]:
mcs = mcs()
mcs.modulation = 'QPSK'
mcs.CRC = '16'
LTB = 16
mcs.Setup(BGN, Z, LTB, maxIter)

In [77]:
mcs.In = np.random.randint(2, size = (mcs.A,1))
tbIn = nrCRCencode(mcs.In,mcs.CRC)
enc = nrLDPCencode(tbIn,mcs)
chIn = nrSymbolModulate(enc,mcs.modulation)
mcs.SignalSize = chIn.shape
mcs.noiseVar = 10**(-snrdB/10)
mcs.noiseStd = 10**(-0.5*snrdB/10)

In [78]:
# NumErrors = np.zeros((NumIter,1),dtype='bool')
# for i in range(0,NumIter):
#     NumErrors[i] = Indicator(chIn, mcs)
# print(np.sum(NumErrors)/NumIter)

In [79]:
MC_buffer = 'result/SNR_' + str(snrdB) + '_BG_' + str(BGN) + '_Z_' + str(Z) + '_buffer.mat'
MC_file = 'result/SNR_' + str(snrdB) + '_BG_' + str(BGN) + '_Z_' + str(Z) + '.mat'
if loadbuffer and os.path.exists(MC_buffer):
    MatData = loadmat(MC_buffer)
    N = MatData['N']
    Ne = MatData['Ne']
    count = MatData['count']
else:
    count = 0
    N = 0
    Ne = 0    

In [80]:
start_time = time.time()

while True:
    rngSeeds = range(count*NumIter,(count+1)*NumIter)
    NumErrors = Parallel(n_jobs=-1)(delayed(Indicator)(i,chIn,mcs) for i in rngSeeds)
    cumError = np.cumsum(NumErrors)
    if (Ne + cumError[-1] < 100):
        Ne += cumError[-1]
    else:
        N = np.where(cumError+Ne == 100)[0][0]
        break
    count += 1
    saveData = {'Ne':Ne, 'N':N, 'count':count}
    savemat(MC_buffer, saveData)

IterMC = N + count*NumIter
PeMC = 100 / IterMC

end_time = time.time()

Runtime = end_time - start_time

saveData = {'N':N, 'PeMC':PeMC, 'Runtime':Runtime}
savemat(MC_file, saveData)